**OpenStreetMap Project**

In [1]:
# Import the libraries:

import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import csv
import codecs
import cerberus
import schema
import sqlite3
import pandas as pd
import pprint

In [2]:
# Code to create smaller osm file by taking every 20th element in original file:

#!/usr/bin/env python
# -*- coding: utf-8 -*-


OSM_FILE = "Edmond.osm"  # Replace this with your osm file
SAMPLE_FILE = "Edmond_sample.osm"

k = 20 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

            
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [3]:
# Count tags in osm file:

def count_tags(filename):
    tags_dict = {}
    for event, elem in ET.iterparse(filename):
        try:
            if elem.tag in tags_dict:
                tags_dict[elem.tag] += 1
            else:
                tags_dict[elem.tag] = 1
        except:
            pass
    
    return tags_dict
    
tags = count_tags('Edmond.osm')
pprint.pprint(tags)

{'bounds': 1,
 'member': 14233,
 'meta': 1,
 'nd': 769842,
 'node': 696135,
 'note': 1,
 'osm': 1,
 'relation': 655,
 'tag': 324255,
 'way': 58104}


In [5]:
# Modify original osm file by creating new tags 'addr:street' and 'addr:postcode' in the 'tiger'-type elements
# where the street name is contained as the value in the tag with the key = 'name', and postcode as the value
# in the tag with the key = 'tiger:zip_left'
# New file is called 'Edmond.xml'

# REFERENCE: https://stackoverflow.com/questions/50998718/take-2-xml-elements-and-merge-into-1-new-element-python

osmfile = 'Edmond.osm'
tree = ET.parse(osmfile)

expected = ["Avenue", "Boulevard", "Circle", "Commons", "Court", "Drive", "Lane", "Parkway", "Place", "Road", "Square", "Street", 
            "Trail", "Terrace", "Turnpike", "Way"]

doc = tree.getroot()
for way in doc.findall(".//way"):
    wname = way.find('.//tag[@k="name"]')
    if wname != None:
        newName = wname.attrib["v"]
        for e in expected:
            if e in newName:
                if way.find('.//tag[@k="addr:street"]') is None:
                    newNode = ET.SubElement(way, 'tag k="addr:street" v="{}"'.format(newName))
        wpcode = way.find('.//tag[@k="tiger:zip_left"]')
        
        if wpcode != None:
            newPcode = wpcode.attrib["v"]
        
            if way.find('.//tag[@k="addr:postcode"]') is None:
                newNode = ET.SubElement(way, 'tag k="addr:postcode" v="{}"'.format(newPcode))
            
for node in doc.findall(".//node"):
    nname = node.find('.//tag[@k="name"]')
    if nname != None:
        newName = nname.attrib["v"]
        for e in expected:
            if e in newName:
                if node.find('.//tag[@k="addr:street"]') is None:
                    newNode = ET.SubElement(node, 'tag k="addr:street" v="{}"'.format(newName))
        npcode = node.find('.//tag[@k="tiger:zip_left"]')
        if npcode != None:
            newPcode = npcode.attrib["v"]
            if node.find('.//tag[@k="addr:postcode"]') is None:
                newNode = ET.SubElement(node, 'tag k="addr:postcode" v="{}"'.format(newPcode))

tree.write(r"Edmond.xml")

In [6]:
# Code to find and count key types for elements in the file:

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":

        if lower.search(element.attrib["k"]):
            keys["lower"] += 1
             
        elif lower_colon.search(element.attrib["k"]):
            keys["lower_colon"] += 1
            
        elif problemchars.search(element.attrib["k"]):
            keys["problemchars"] += 1
            
        else:
            keys["other"] += 1
            
    return keys
            

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = process_map('Edmond.xml')
pprint.pprint(keys)

{'lower': 157235, 'lower_colon': 122886, 'other': 65185, 'problemchars': 0}


In [7]:
# Code to count all users contributing to creation of the file:

def get_user(element):
    return

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        user = 'uid'
        if user in element.attrib:
            users.add(element.attrib[user])
    return users

users = process_map('Edmond.xml')
print len(users)

515


In [10]:
# THIS IS THE MAIN CODE IN THE PROJECT
# Combine codes audit.py and data.py from Udacity Case Study to audit and clean the data and write to csv files:

# REFERENCE: https://libraries.io/github/mkuehn10/P3-Wrangle-OpenStreetMap-Data

OSMFILE = "Edmond.xml"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
postcode_type_re = re.compile(r'.*(\d{5}(\-\d{4})?)$')

postcodes = set()                            

expected = ["Avenue", "Boulevard", "Circle", "Commons", "Court", "Drive", "Lane", "Parkway", "Place", "Road", "Square", "Street", 
            "Trail", "Terrace", "Way", "North", "Northwest", "Northeast", "South", "Southwest", "Southeast", "East", "West"]


mapping = { "St": "Street",
            "St.": "Street",
            "STREET": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Pl": "Place",
            "Terr": "Terrace",
            "N": "North",
            "NW": "Northwest",
            "NE": "Northeast",
            "S": "South",
            "SW": "Southwest",
            "SE": "Southeast",
            "E": "East",
            "W": "West"
        }


def audit_street_type(street_types, street_name):
    
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit(osmfile):
    
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    postcode_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                elif is_postcode(tag):
                    postcodes.add(tag.attrib['v'])
                
    osm_file.close()
    return street_types


def clean_street_name(name, mapping):
    better_name = []
    for split_name in name.split(' '):
        if split_name in mapping.keys():
            split_name = mapping[split_name]
        better_name.append(split_name)
    
    better_name = ' '.join(better_name)
    return better_name

#st_types = audit(OSMFILE)
#for st_type, ways in st_types.iteritems():
#    for name in ways:
#        better_name = clean_street_name(name, mapping)
#        print name, "=>", better_name


NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.Schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    if element.tag == 'node':
        for attrib in NODE_FIELDS:
            if attrib in element.attrib:
                node_attribs[attrib] = element.attrib[attrib]
            
        for sec in element.iter("tag"):
            if sec.attrib["k"] == "addr:street":
                sec.attrib["v"] = clean_street_name(sec.attrib["v"], mapping) 
            if sec.attrib["k"] == "addr:postcode":
                sec.attrib["v"] = sec.attrib["v"][:5]
            tag_dict = {'id': None, 'key': None, 'value': None, 'type': None}
            tag_dict["id"] = element.attrib["id"]
            tag_dict["value"] = sec.attrib["v"]
            tag_dict["type"] = default_tag_type
            k_v = sec.attrib["k"]

            if ':' in k_v:
                type, key = k_v.split(':', 1)
                tag_dict["key"] = key
                tag_dict["type"] = type
            else:
                tag_dict["key"] = k_v
            tags.append(tag_dict)
        
    
    if element.tag == "way":
        for attrib in WAY_FIELDS:
            if attrib in element.attrib:
                way_attribs[attrib] = element.attrib[attrib]
                
        for c, tag_dict in enumerate(element.iter("nd")):
            way_node = {"id": None, "node_id": None, "position": 0}
            way_node["id"] = element.attrib["id"]
            way_node["node_id"] = tag_dict.attrib["ref"]
            way_node["position"] = c
            way_nodes.append(way_node)
        
        for sec in element.iter("tag"):
            if sec.attrib["k"] == "addr:street":
                sec.attrib["v"] = clean_street_name(sec.attrib["v"], mapping) 
            if sec.attrib["k"] == "addr:postcode":
                sec.attrib["v"] = sec.attrib["v"][:5] 
            
            tag_dict = {'id': None, 'key': None, 'value': None, 'type': None}
            tag_dict["id"] = element.attrib["id"]
            tag_dict["value"] = sec.attrib["v"]
            tag_dict["type"] = default_tag_type
            k_v = sec.attrib["k"]
            
            if ':' in k_v:
                type, key = k_v.split(':', 1)
                tag_dict["key"] = key
                tag_dict["type"] = type
            else:
                tag_dict["key"] = k_v
            tags.append(tag_dict)
        
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


# ERROR DURING VALIDATION
            
#def validate_element(element, validator, schema=SCHEMA):
#    """Raise ValidationError if element does not match schema"""
#    if validator.validate(element, schema) is not True:
#        field, errors = next(validator.errors.iteritems())
#        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
#        error_string = pprint.pformat(errors)
#        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
#                if validate is True:
#                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


# Note: Validation is ~ 10X slower. For the project consider using a small
# sample of the map when validating.
process_map(OSMFILE, validate=True)

Circle Glen => Circle Glen
Memorial Road Church of Christ => Memorial Road Church of Christ
BLUE JAY DRIVE => BLUE JAY DRIVE
West Danforth => West Danforth
W Britton Rd => West Britton Road
W. Covell Rd => W. Covell Road
W I-35 Frontage Rd => West I-35 Frontage Road
W Covell Rd => West Covell Road
West Memorial Rd => West Memorial Road
W Danforth Rd => West Danforth Road
West I 35 Frontage Rd => West I 35 Frontage Road
W Memorial Rd => West Memorial Road
Highland Trails Baseball Field => Highland Trails Baseball Field
NW `155TH STREET => Northwest `155TH Street
NW 156TH STREET => Northwest 156TH Street
NW 164TH STREET => Northwest 164TH Street
NW 155TH STREET => Northwest 155TH Street
NW 150TH STREET => Northwest 150TH Street
Highland Trails Playground => Highland Trails Playground
East Highway 66 => East Highway 66
Highway 66 => Highway 66
West Lake Hefner Dr. => West Lake Hefner Dr.
Road Not Taken => Road Not Taken
Northwest Expressway => Northwest Expressway
Hilltop Drive (S3296 Rd)

In [49]:
# Create connection with the database and execute some SQL queries:
# REFERENCE: 1) https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md;
# 2) https://stackoverflow.com/questions/7831371/is-there-a-way-to-get-a-list-of-column-names-in-sqlite

connection = sqlite3.connect('EdmondDB.db')
c = connection.cursor()

#List top 10 contributors for the OSM data:

QUERY = '''
SELECT e.user as User, COUNT(*) as Count
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user ORDER BY COUNT(*) DESC;
'''

cur = c.execute(QUERY)

names = list(map(lambda x: x[0], cur.description))
names = [description[0] for description in cur.description]

rows = c.fetchall()

df = pd.DataFrame(rows)
df.columns = names
print df.head(10)


              User   Count
0          OklaNHD  332971
1     dustybrimaps   97134
2     Paul Johnson   59853
3    JamesTheElder   51602
4  William McBroom   32407
5  woodpeck_fixbot   23368
6          dufekin   22122
7          JoeCat1   12240
8            Rub21    8447
9          mhenson    7538


In [32]:
QUERY = '''
SELECT e.value as Amenity, COUNT(e.value) as Count
FROM (SELECT key, value FROM nodes_tags WHERE nodes_tags.key='amenity'
UNION ALL SELECT key, value FROM ways_tags
WHERE ways_tags.key='amenity') e
GROUP BY e.value ORDER BY COUNT(e.value) DESC;
'''
cur = c.execute(QUERY)

names = list(map(lambda x: x[0], cur.description))
names = [description[0] for description in cur.description]

rows = c.fetchall()

df = pd.DataFrame(rows)
df.columns = names
print df.head(5)


            Amenity  Count
0           parking    785
1  place_of_worship    166
2        restaurant    125
3         fast_food    118
4            school    109


In [29]:
QUERY = '''
SELECT e.value as Church, COUNT(e.value) as Count
FROM (SELECT value FROM nodes_tags WHERE key='denomination'
UNION ALL SELECT value FROM ways_tags WHERE ways_tags.key = 'denomination') e
GROUP BY e.value ORDER BY COUNT(e.value) DESC, e.value;
'''
cur = c.execute(QUERY)

names = list(map(lambda x: x[0], cur.description))
names = [description[0] for description in cur.description]

rows = c.fetchall()

df = pd.DataFrame(rows)
df.columns = names
print df


                   Church  Count
0                 baptist     29
1                lutheran      8
2               methodist      6
3            presbyterian      5
4                catholic      4
5             pentecostal      4
6        church_of_christ      3
7                  mormon      3
8          roman_catholic      3
9             evangelical      2
10  seventh_day_adventist      2
11               anglican      1
12         greek_orthodox      1
13      nondenominational      1
14               orthodox      1
15             protestant      1
16       southern_baptist      1
17     ukrainian_orthodox      1


In [30]:
QUERY = '''
SELECT e.value as Religion, COUNT(e.value) as Count
FROM (SELECT value FROM nodes_tags WHERE key='religion'
UNION ALL SELECT value FROM ways_tags WHERE ways_tags.key = 'religion') e
GROUP BY e.value ORDER BY COUNT(e.value) DESC, e.value;
'''
cur = c.execute(QUERY)

names = list(map(lambda x: x[0], cur.description))
names = [description[0] for description in cur.description]

rows = c.fetchall()

df = pd.DataFrame(rows)
df.columns = names
print df


                 Religion  Count
0               christian    161
1                  muslim      1
2  unitarian_universalist      1


In [35]:
QUERY = '''
SELECT e.value as Cuisine, COUNT(e.value) as Count
FROM (SELECT value FROM nodes_tags WHERE key='cuisine'
UNION ALL SELECT value FROM ways_tags WHERE ways_tags.key = 'cuisine') e
GROUP BY e.value ORDER BY COUNT(e.value) DESC, e.value;
'''
cur = c.execute(QUERY)

names = list(map(lambda x: x[0], cur.description))
names = [description[0] for description in cur.description]

rows = c.fetchall()

df = pd.DataFrame(rows)
df.columns = names
print df.head(10)


     Cuisine  Count
0     burger     46
1    mexican     16
2      pizza     15
3   sandwich     14
4    chicken     10
5   american      8
6    chinese      7
7  ice_cream      7
8    tex-mex      6
9      asian      5


In [68]:
QUERY1 = '''
SELECT COUNT(DISTINCT(e.value)) as Elementary
FROM (SELECT value FROM nodes_tags WHERE key='name' AND value LIKE '%Elementary School'
UNION ALL SELECT value FROM ways_tags WHERE key = 'name' AND value LIKE '%Elementary School') e
ORDER BY e.value;
'''
QUERY2 = '''
SELECT COUNT(DISTINCT(e.value)) as Middle
FROM (SELECT value FROM nodes_tags WHERE key='name' AND value LIKE '%Middle School'
UNION ALL SELECT value FROM ways_tags WHERE key = 'name' AND value LIKE '%Middle School') e
ORDER BY e.value;
'''
QUERY3 = '''
SELECT COUNT(DISTINCT(e.value)) as High
FROM (SELECT value FROM nodes_tags WHERE key='name' AND value LIKE '%High School'
UNION ALL SELECT value FROM ways_tags WHERE key = 'name' AND value LIKE '%High School') e
ORDER BY e.value;
'''
rows1 = c.execute(QUERY1).fetchall()
rows2 = c.execute(QUERY2).fetchall()
rows3 = c.execute(QUERY3).fetchall()
print('Edmond elementary schools: ' + str(rows1[0][0]))
print('Edmond middle schools: ' + str(rows2[0][0]))
print('Edmond junior high/high schools: ' + str(rows3[0][0]))


Edmond elementary schools: 18
Edmond middle schools: 6
Edmond junior high/high schools: 14


In [73]:
QUERY = '''
SELECT e.value as Sport, COUNT(e.value) as Count
FROM (SELECT value FROM nodes_tags WHERE key='sport'
UNION ALL SELECT value FROM ways_tags WHERE ways_tags.key = 'sport') e
GROUP BY e.value ORDER BY COUNT(e.value) DESC, e.value;
'''
cur = c.execute(QUERY)

names = list(map(lambda x: x[0], cur.description))
names = [description[0] for description in cur.description]

rows = c.fetchall()

df = pd.DataFrame(rows)
df.columns = names
print df


                Sport  Count
0              tennis    112
1            baseball     95
2                golf     93
3              soccer     59
4          basketball     41
5            swimming     31
6   american_football     13
7          volleyball      8
8     beachvolleyball      5
9               10pin      2
10            fitness      2
11           football      2
12              motor      1
13         skateboard      1
14               yoga      1


In [97]:
QUERY = '''
SELECT COUNT(e.Name) as Number, e.Name, e.Type FROM
(SELECT c.value as Type, b.value as Name, a.value as Cuisine 
FROM nodes_tags as a, nodes_tags as b, nodes_tags as c
WHERE a.id = b.id AND a.id = c.id AND a.key = 'cuisine' AND a.type = 'regular' AND b.key = 'name' AND b.type = 'regular' AND c.key = 'amenity' AND c.type = 'regular'
UNION ALL SELECT c.value as Type, b.value as Name, a.value as Cuisine 
FROM ways_tags as a, ways_tags as b, ways_tags as c
WHERE a.id = b.id AND a.id = c.id AND a.key = 'cuisine' AND a.type = 'regular' AND b.key = 'name'  AND b.type = 'regular' AND c.key = 'amenity' AND c.type = 'regular') e
GROUP BY e.Name ORDER BY Count(e.Name) DESC, e.Type, e.Name;
'''
cur = c.execute(QUERY)

names = list(map(lambda x: x[0], cur.description))
names = [description[0] for description in cur.description]

rows = c.fetchall()

df = pd.DataFrame(rows)
df.columns = names
print df

    Number                            Name        Type
0       14                           Sonic   fast_food
1       12                      McDonald's   fast_food
2        7                         Braum's   fast_food
3        5                     Chick-fil-A   fast_food
4        5                       Taco Bell   fast_food
5        5                     Whataburger   fast_food
6        4                       Starbucks        cafe
7        4                          Arby's   fast_food
8        4                            IHOP  restaurant
9        3                             KFC   fast_food
10       3                   Panda Express   fast_food
11       3                          Subway   fast_food
12       2                     Burger King   fast_food
13       2                      Carl's Jr.   fast_food
14       2                        Chipotle   fast_food
15       2                 Daylight Donuts   fast_food
16       2                  Little Caesars   fast_food
17       2

In [98]:
connection.close()